In [ ]:
#-------------------------------------------------------------------
# Author Wessel Olaf van Dam 04-10-2019
#
# script to analyze mean RTs in Emotion regulation study
#-------------------------------------------------------------------

print("This is an analysis of RTs for Emotion Regulation study")
print ("\n")

# in the following lines we will import pandas to easily read in our excel data file

import os
import pandas as pd
import numpy as np

pm = pd.read_excel('ALLSUBJECTS.xls')
# listing all your column headers so we can select the relevant ones 
#list(pm.columns.values) 

# cleaning up the dataframe by selecting the columns of interest
data = pm[['stimulation','Subject','PicValence', 'Rating','Ratepicimage.RESP','Ratepicimage.RT','RegInst']]
df = pd.DataFrame(data)

# here we add a column to our Dataframe that will tell us to which condition the trial belonged

def condition (row):
   if row['stimulation'] == "PRESTIM" and row['PicValence'] == "negative" and row['RegInst'] == "decrease"  :
      return 1
   if row['stimulation'] == "STIM" and row['PicValence'] == "negative" and row['RegInst'] == "decrease"  :
      return 2   
   if row['stimulation'] == "PRESTIM" and row['PicValence'] == "negative" and row['RegInst'] == "look"  :
      return 3
   if row['stimulation'] == "STIM" and row['PicValence'] == "negative" and row['RegInst'] == "look"  :
      return 4    
   if row['stimulation'] == "PRESTIM" and row['PicValence'] == "neutral" and row['RegInst'] == "look"  :
      return 5
   if row['stimulation'] == "STIM" and row['PicValence'] == "neutral" and row['RegInst'] == "look"  :
      return 6    
   if row['stimulation'] == "PRESTIM" and row['PicValence'] == "positive" and row['RegInst'] == "decrease"  :
      return 7
   if row['stimulation'] == "STIM" and row['PicValence'] == "positive" and row['RegInst'] == "decrease"  :
      return 8   
   if row['stimulation'] == "PRESTIM" and row['PicValence'] == "positive" and row['RegInst'] == "look"  :
      return 9
   if row['stimulation'] == "STIM" and row['PicValence'] == "positive" and row['RegInst'] == "look"  :
      return 10
    
condition = df.apply (lambda row: condition(row), axis=1)
new_column = pd.DataFrame(condition)
new_column.columns = ["Condition"]
data = pd.concat([df,new_column], axis =1)

data = data.dropna() #dropping NaN responses
print (data.head(20))

# to check if the new column was added succesfully we check the first 5 lines of the new Dataframe
#print data.head(5)

# here we use a dictionary to select a subset of the dataframe for each of our subjects

subjectlist={}
for x in range(1,38):
     subjectlist["Subject{0}".format(x)]= data[data.Subject == x]

#print (subjectlist["Subject1"])

def rem_outliers3(aseries, nsd, method):
    if method == 'sd':
        m = aseries.mean()
        s = aseries.std()
        upper = m + nsd*s
        lower = m - nsd*s
        newseries = cond[(aseries < upper) & (aseries > lower)]
        return newseries
    if method == 'iqr':
        med = aseries.median()
        q1 = aseries.quantile(0.25)
        q3 = aseries.quantile(0.75)
        iqr = (q3-q1)
        low = med - iqr*nsd
        up = med + iqr*nsd
        newseries2 = cond[(aseries < up) & (aseries > low)]
        return newseries2
    if method != 'iqr' or 'sd':
        print('Warning sd will be applied')
        m = aseries.mean()
        s = aseries.std()
        upper = m + nsd*s
        lower = m - nsd*s
        newseries = cond[(aseries < upper) & (aseries > lower)]
        return newseries
    
conditionlist={}
for x in range(1,38):
    temp = subjectlist["Subject{0}".format(x)] # this will contain all the 275 trials for subj 2,3..etc
    for y in range(1,11):
        conditionlist["Condition{0}".format(y)] = temp[temp.Condition == y] #this will contain all the 275 trials
        cond = temp[temp.Condition == y]
        outliers_removed = rem_outliers3(cond['Ratepicimage.RT'], 2, 'sd')
        df = pd.DataFrame(outliers_removed)
        conditionlist["Condition{0}".format(y)] = df #this will contain only the non-outlier trials
        #print conditionlist
    subjectlist["Subject{0}".format(x)] = pd.concat(conditionlist.values(), ignore_index=True) #subj list with only non-outlier trials

    
# here we put everything back together for a clean pm
pmclean = pd.concat(subjectlist.values(), ignore_index=True)
print (pmclean.head(20))

In [ ]:
#print pmclean

# here we use a dictionary to select a subset of the dataframe and calculate mean RTs
# for each condition and add it to an empty dataframe. Each row corresponds to a subject
# each column corresponds to a condition

subjectlist={}
conditionlist={}
# here we create an empty Dataframe with 37 rows and 5 columns
result = pd.DataFrame(np.zeros((37, 10)))

for x in range(1,38):
    subjectlist["Subject{0}".format(x)]= pmclean[pmclean.Subject == x]
    temp = subjectlist["Subject{0}".format(x)]
    
    for y in range(1,11):
        conditionlist["cond{0}".format(y)] = temp[temp.Condition == y] 
        cond = temp[temp.Condition == y]
        meanRT = cond['Ratepicimage.RT'].mean()   
        result.loc[x-1,y-1] = int(meanRT)

# now we have a Dataframe with all the results for all subjects

#print result

# Here we rename the columns and rows so we have labels for conditions and subjects 

result.rename(columns={0:'NegDec_PRESTIM RT',1:'NegDec_STIM RT',2:'NegLook_PRESTIM RT',3:'NegLook_STIM RT',4:'NeuLook_PRESTIM RT',5:'NeuLook_STIM RT',
                      6:'PosDec_PRESTIM RT',7:'PosDec_STIM RT',8:'PosLook_PRESTIM RT',9:'PosLook_STIM RT'},inplace=True)
result.rename(index={0:'Subj1',1:'Subj2',2:'Subj3',3:'Subj4',4:'Subj5',5:'Subj6',6:'Subj7',7:'Subj8',8:'Subj9',9:'Subj10',
                    10:'Subj11',11:'Subj12',12:'Subj13',13:'Subj14',14:'Subj15',15:'Subj16',16:'Subj17',17:'Subj18',
                     18:'Subj19',19:'Subj20',20:'Subj21',21:'Subj22',22:'Subj23',23:'Subj24',24:'Subj25',25:'Subj26',
                     26:'Subj27',27:'Subj28',28:'Subj29',29:'Subj30',30:'Subj31',31:'Subj32',32:'Subj33',33:'Subj34',
                     34:'Subj35',35:'Subj36',36:'Subj37'
                    },inplace=True)

print(result)

# here we store the dataframe in an excelfile 

result.to_excel("RTs_EmotionRegulation.xlsx")

 
